In [158]:
!pip install xgboost
!pip install catboost

python(64548) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python(64563) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [160]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna.integration.lightgbm as lgb
import xgboost as xgb


from catboost import CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor


# 準備

In [111]:
"""データの読込"""
# 欠損値ありデータ
# train_df = pd.read_csv("../data/train.csv", index_col=0)
# test_df  = pd.read_csv("../data/test.csv", index_col=0)

# 欠損値なしデータ
train_df = pd.read_csv("../data/train_imputed_with_lgbm_new_feat.csv", index_col=0)
test_df  = pd.read_csv("../data/test_imputed_with_lgbm_new_feat.csv", index_col=0)

test_df.head()

,purpose,int.rate,installment,annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,purpose.kmeans_3,purpose.all_other_cluster,purpose.all_other_cluster.kmeans_3,inq.delinq.pub_risk,inq.delinq.pub_risk.kmeans_3,high_int.rate
40785,2,0.1543,225.68,16525.86,22.39,668,1960,43771.0,59.3,3,0,0,2,4,0,3,0,0
40786,2,0.1539,200.43,62412.73,21.84,660,5958,13607.0,87.0,1,1,0,2,4,0,2,0,0
40787,2,0.1503,252.35,42836.79,21.77,663,4950,1832.0,36.8,5,0,1,2,4,0,6,0,0
40788,2,0.1316,533.85,27821.44,10.60,690,1825,3597.0,60.0,0,0,0,2,4,0,0,0,0
40789,2,0.1506,484.50,91406.09,17.35,658,3253,12137.0,48.8,1,0,0,2,4,0,1,0,0


In [115]:
"""データの基本情報"""

print(f"訓練データセットの形状： {train_df.shape}")
print(f"テストデータセットの形状： {test_df.shape}")

print()
print("===基本統計量===")
print("<訓練データセット>")
display(train_df.describe())
print("<最終テストデータセット>")
display(test_df.describe())


print()
print("===要約情報===")
print("<訓練データセット>")
display(train_df.info())
print("<最終テストデータセット>")
display(test_df.info())

訓練データセットの形状： (40785, 19)
テストデータセットの形状： (40786, 18)

===基本統計量===
<訓練データセット>


,purpose,int.rate,installment,annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid,purpose.kmeans_3,purpose.all_other_cluster,purpose.all_other_cluster.kmeans_3,inq.delinq.pub_risk,inq.delinq.pub_risk.kmeans_3,high_int.rate
count,40785.000000,40785.000000,40785.000000,4.078500e+04,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000,40785.000000
mean,2.747628,0.125414,302.319235,7.087001e+04,12.530579,709.393061,4736.481746,15426.025468,46.161957,1.543018,0.180998,0.039696,0.151870,1.457399,4.511879,0.690916,1.763712,0.154003,0.000123
std,1.986736,0.030167,208.058959,6.524275e+04,6.831929,39.299297,2568.216636,22343.495560,28.258810,2.059082,0.513544,0.195246,0.358899,0.715350,2.378566,0.878474,2.187254,0.360956,0.011072
min,0.000000,0.060800,15.680000,2.086720e+03,0.110000,615.000000,179.000000,1.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.105300,157.400000,3.868640e+04,6.700000,679.000000,2965.000000,3433.000000,22.500000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,0.125200,241.120000,5.862029e+04,12.740000,703.000000,4304.000000,8441.000000,44.500000,1.000000,0.000000,0.000000,0.000000,2.000000,4.000000,0.000000,1.000000,0.000000,0.000000
75%,4.000000,0.145100,391.990000,8.416863e+04,17.930000,737.000000,5896.000000,19085.000000,68.900000,2.000000,0.000000,0.000000,0.000000,2.000000,6.000000,2.000000,3.000000,0.000000,0.000000
max,6.000000,0.800000,939.380000,1.913363e+06,29.930000,826.000000,16888.000000,356328.000000,108.300000,14.000000,3.000000,1.000000,1.000000,2.000000,8.000000,2.000000,17.000000,1.000000,1.000000


<最終テストデータセット>


,purpose,int.rate,installment,annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,purpose.kmeans_3,purpose.all_other_cluster,purpose.all_other_cluster.kmeans_3,inq.delinq.pub_risk,inq.delinq.pub_risk.kmeans_3,high_int.rate
count,40786.000000,40786.00000,40786.000000,4.078600e+04,40786.000000,40786.000000,40786.000000,40786.000000,40786.00000,40786.00000,40786.000000,40786.000000,40786.000000,40786.000000,40786.000000,40786.000000,40786.000000,40786.000000
mean,2.758888,0.12544,300.571366,7.019006e+04,12.552485,709.242338,4685.578556,15249.182289,46.41177,1.52876,0.179694,0.038739,1.459520,4.524616,0.689918,1.747193,0.151425,0.000172
std,1.980095,0.03087,208.171846,6.145846e+04,6.836684,39.320390,2540.021093,22272.133847,28.32093,2.08263,0.513398,0.192974,0.713822,2.372009,0.878961,2.207671,0.358467,0.013100
min,0.000000,0.06160,15.680000,2.128340e+03,0.090000,616.000000,181.000000,1.000000,0.10000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.10540,155.790000,3.855412e+04,6.700000,679.000000,2928.250000,3367.000000,23.00000,0.00000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,0.12510,240.400000,5.814888e+04,12.760000,704.000000,4253.000000,8258.000000,44.70000,1.00000,0.000000,0.000000,2.000000,4.000000,0.000000,1.000000,0.000000,0.000000
75%,4.000000,0.14510,388.415000,8.379708e+04,17.950000,736.750000,5834.000000,18686.750000,69.10000,2.00000,0.000000,0.000000,2.000000,6.000000,2.000000,3.000000,0.000000,0.000000
max,6.000000,0.90000,938.860000,2.000790e+06,29.950000,826.000000,17416.000000,296156.000000,109.60000,14.00000,3.000000,1.000000,2.000000,8.000000,2.000000,16.000000,1.000000,1.000000



===要約情報===
<訓練データセット>
<class 'pandas.core.frame.DataFrame'>
Int64Index: 40785 entries, 0 to 40784
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   purpose                             40785 non-null  int64  
 1   int.rate                            40785 non-null  float64
 2   installment                         40785 non-null  float64
 3   annual.inc                          40785 non-null  float64
 4   dti                                 40785 non-null  float64
 5   fico                                40785 non-null  int64  
 6   days.with.cr.line                   40785 non-null  int64  
 7   revol.bal                           40785 non-null  float64
 8   revol.util                          40785 non-null  float64
 9   inq.last.6mths                      40785 non-null  int64  
 10  delinq.2yrs                         40785 non-null  int64  
 11  pub.rec           

None

<最終テストデータセット>
<class 'pandas.core.frame.DataFrame'>
Int64Index: 40786 entries, 40785 to 81570
Data columns (total 18 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   purpose                             40786 non-null  int64  
 1   int.rate                            40786 non-null  float64
 2   installment                         40786 non-null  float64
 3   annual.inc                          40786 non-null  float64
 4   dti                                 40786 non-null  float64
 5   fico                                40786 non-null  int64  
 6   days.with.cr.line                   40786 non-null  int64  
 7   revol.bal                           40786 non-null  float64
 8   revol.util                          40786 non-null  float64
 9   inq.last.6mths                      40786 non-null  int64  
 10  delinq.2yrs                         40786 non-null  int64  
 11  pub.rec                

None

# 前処理

### 関数

In [114]:
def log_transform(x):
    return np.log(x)

In [74]:
"""欠損値のある行の削除"""
print(f"欠損値前の訓練/テストデータの形状：{train_df.shape}")

# 欠損値の削除
train_df_drop = train_df.dropna()

print(f"欠損値後の訓練/テストデータの形状：{train_df_drop.shape}")

欠損値前の訓練/テストデータの形状：(40785, 16)
欠損値後の訓練/テストデータの形状：(40785, 16)


In [6]:
"""テストデータの欠損値は中央値で補完"""

test_df["purpose"].fillna(test_df["purpose"].mode()[0], inplace=True)
test_df['installment'].fillna(test_df["installment"].median(), inplace=True)
test_df['revol.bal'].fillna(test_df["revol.bal"].median(), inplace=True)
test_df['revol.util'].fillna(test_df["revol.util"].median(), inplace=True)

In [242]:
"""カテゴリカル変数（purpose）のエンコーディング"""
# "purpose"列をone-hotエンコーディング

col_to_one = ['purpose', 'purpose.kmeans_3', 'purpose.all_other_cluster', 'purpose.all_other_cluster.kmeans_3', "inq.delinq.pub_risk", "inq.delinq.pub_risk.kmeans_3"]
col_to_log = ["annual.inc", "revol.bal"]
col_to_scl = ["installment", "int.rate", "dti", "fico", "days.with.cr.line", "revol.util"]

# 目的変数を除いたデータフレームを作成
X_train = train_df.drop(columns=['not.fully.paid'])
y_train = train_df['not.fully.paid']

# ColumnsTransformerを使用してOne-Hotエンコーディングを設定
col_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), col_to_one),
        # ('scale', StandardScaler(), col_to_scl),
        # ('log', FunctionTransformer(log_transform), col_to_log)
    ],
    remainder="passthrough", # 変換しない列はそのまま通す
)

# 訓練データのエンコード
train_encoded = col_transformer.fit_transform(X_train)
# テストデータのエンコード
test_encoded = col_transformer.transform(test_df)

# エンコードされたデータをデータフレームに変換
new_columns = []
for name, transformer, columns in col_transformer.transformers_:
    if name == 'remainder':
        new_columns.extend(columns)
    else:
        if hasattr(transformer, 'get_feature_names_out'):
            new_columns.extend(transformer.get_feature_names_out(columns))
        else:
            new_columns.extend(columns)

train_df_encoded = pd.DataFrame(train_encoded, columns=new_columns)
test_df_encoded = pd.DataFrame(test_encoded, columns=new_columns)

# エンコード後の訓練データフレームに目的変数を追加
train_df_encoded['not.fully.paid'] = y_train.values

In [243]:
"""データの分割"""

# 説明変数/目的変数の分割
X = train_df_encoded.drop(columns=["not.fully.paid"])
y = train_df_encoded["not.fully.paid"]

# 訓練/テストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [1]:
train_df_encoded

NameError: name 'train_df_encoded' is not defined

# モデルの構築

## LightGMB

In [245]:
"""GridSearchCVでパラメータチューニングを行う"""

# チューニングするパラメータ
params = {
    "n_estimators": [280],
    "num_leaves": [7],
    "learning_rate": [0.05],
    "lambda_l1": [0.7],
    "lambda_2": [0],
    "verbosity": [-1],
}

# インスタンスの作成
lgbm = LGBMClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=params,
    cv=10,
    scoring="roc_auc"
)

# モデルの訓練
grid_search.fit(X_train, y_train)
print(f"最適なパラメータ: {grid_search.best_params_}")

best_lgbm = grid_search.best_estimator_

# 予測確率の計算
y_pred_lgbm = best_lgbm.predict_proba(X_test)[:, 1]

最適なパラメータ: {'lambda_2': 0, 'lambda_l1': 0.7, 'learning_rate': 0.05, 'n_estimators': 280, 'num_leaves': 7, 'verbosity': -1}


In [246]:
# 閾値0.6を使用して予測ラベルを生成
threshold = 0.2
y_pred_custom_threshold = (y_pred_lgbm >= threshold).astype(int)

# AUCスコアの計算
custom_auc = roc_auc_score(y_test, y_pred_custom_threshold)

# 結果の表示
print(f"AUC Score with custom threshold 0.5: {custom_auc:.2f}")
print("\nClassification Report with custom threshold 0.5:")
print(classification_report(y_test, y_pred_custom_threshold))

AUC Score with custom threshold 0.5: 0.72

Classification Report with custom threshold 0.5:
              precision    recall  f1-score   support

           0       0.93      0.80      0.86     10378
           1       0.36      0.64      0.46      1858

    accuracy                           0.78     12236
   macro avg       0.65      0.72      0.66     12236
weighted avg       0.84      0.78      0.80     12236



## XGboost

In [247]:
""""""

# DMatrixの作成
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# パラメータの設定
params = {
    'n_estimators': [250],
    'learning_rate': [0.05],
    'max_depth': [5],
    'colsample_bytree': [0.3],
    "gamma": [0.1],
    "alpha": [5.0],
    "lambda": [1.0],
}

xgb_model = xgb.XGBRegressor(random_state=42)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=params,
    cv=5,
    scoring="roc_auc",
    verbose=1
)

# モデルの訓練
grid_search.fit(X_train, y_train)
print(f"最適なパラメータ: {grid_search.best_params_}")


best_xgb = grid_search.best_estimator_

# 予測確率の計算
y_pred_xgb = best_xgb.predict(X_test)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
最適なパラメータ: {'alpha': 5.0, 'colsample_bytree': 0.3, 'gamma': 0.1, 'lambda': 1.0, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 250}


In [248]:
# 閾値0.6を使用して予測ラベルを生成
threshold = 0.1
y_pred_custom_threshold = (y_pred_xgb >= threshold).astype(int)

# AUCスコアの計算
custom_auc = roc_auc_score(y_test, y_pred_custom_threshold)

# 結果の表示
print(f"AUC Score with custom threshold 0.5: {custom_auc:.2f}")
print("\nClassification Report with custom threshold 0.5:")
print(classification_report(y_test, y_pred_custom_threshold))

AUC Score with custom threshold 0.5: 0.69

Classification Report with custom threshold 0.5:
              precision    recall  f1-score   support

           0       0.96      0.51      0.67     10378
           1       0.24      0.87      0.38      1858

    accuracy                           0.57     12236
   macro avg       0.60      0.69      0.52     12236
weighted avg       0.85      0.57      0.63     12236



## CatBoost

In [249]:
# CatBoostの回帰モデルのインスタンスを作成
catboost = CatBoostRegressor(random_state=42, verbose=0)

# ハイパーパラメータの設定
params = {
    'iterations': [450],
    'learning_rate': [0.05],
    'depth': [5],
    'colsample_bylevel': [1.0],
    'l2_leaf_reg': [10],
    'bagging_temperature': [0.5]
}

# GridSearchCVのインスタンスを作成
grid_search = GridSearchCV(
    estimator=catboost,
    param_grid=params,
    cv=5,
    scoring='neg_mean_squared_error',
    )

# ハイパーパラメータの探索
grid_search.fit(X_train, y_train)

# 最適なパラメータの表示
print(f"最適なパラメータ: {grid_search.best_params_}")

# 最適なモデルを使用して予測
best_cat = grid_search.best_estimator_
y_pred_cat = best_cat.predict(X_test)

最適なパラメータ: {'bagging_temperature': 0.5, 'colsample_bylevel': 1.0, 'depth': 5, 'iterations': 450, 'l2_leaf_reg': 10, 'learning_rate': 0.05}


In [250]:
# 閾値0.6を使用して予測ラベルを生成
threshold = 0.2
y_pred_custom_threshold = (y_pred_cat >= threshold).astype(int)

# AUCスコアの計算
custom_auc = roc_auc_score(y_test, y_pred_custom_threshold)

# 結果の表示
print(f"AUC Score with custom threshold 0.5: {custom_auc:.2f}")
print("\nClassification Report with custom threshold 0.5:")
print(classification_report(y_test, y_pred_custom_threshold))

AUC Score with custom threshold 0.5: 0.72

Classification Report with custom threshold 0.5:
              precision    recall  f1-score   support

           0       0.93      0.78      0.85     10378
           1       0.35      0.67      0.46      1858

    accuracy                           0.76     12236
   macro avg       0.64      0.72      0.65     12236
weighted avg       0.84      0.76      0.79     12236



## HistGradientBoosting

In [251]:
# インスタンスの作成
hgb_reg = HistGradientBoostingRegressor(random_state=42)

# すべてのカラム名を文字列に変換
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)


# ハイパーパラメータの設定
param_grid = {
    'learning_rate': [0.05],
    'max_iter': [300],
    'max_depth': [3,],
    'min_samples_leaf': [20],
    'max_bins': [200],
    'l2_regularization': [1]
}

# GridSearchCVのインスタンスを作成
grid_search = GridSearchCV(
    estimator=hgb_reg,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=1
)

# ハイパーパラメータの探索
grid_search.fit(X_train, y_train)

# 最適なパラメータの表示
print(f"最適なパラメータ: {grid_search.best_params_}")

# 最適なモデルを使用して予測
best_hgb = grid_search.best_estimator_
y_pred_hgb = best_hgb.predict(X_test)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
最適なパラメータ: {'l2_regularization': 1, 'learning_rate': 0.05, 'max_bins': 200, 'max_depth': 3, 'max_iter': 300, 'min_samples_leaf': 20}


In [252]:
# 閾値0.6を使用して予測ラベルを生成
threshold = 0.2
y_pred_custom_threshold = (y_pred_hgb >= threshold).astype(int)

# AUCスコアの計算
custom_auc = roc_auc_score(y_test, y_pred_custom_threshold)

# 結果の表示
print(f"AUC Score with custom threshold 0.5: {custom_auc:.2f}")
print("\nClassification Report with custom threshold 0.5:")
print(classification_report(y_test, y_pred_custom_threshold))

AUC Score with custom threshold 0.5: 0.72

Classification Report with custom threshold 0.5:
              precision    recall  f1-score   support

           0       0.93      0.78      0.85     10378
           1       0.35      0.66      0.45      1858

    accuracy                           0.76     12236
   macro avg       0.64      0.72      0.65     12236
weighted avg       0.84      0.76      0.79     12236



In [253]:
ensemble_pred = (y_pred_lgbm + y_pred_xgb + y_pred_cat + y_pred_hgb) / 4
print(ensemble_pred)

[0.05995865 0.03474087 0.26991274 ... 0.36383005 0.10533439 0.05264684]


In [254]:
# 閾値0.6を使用して予測ラベルを生成
threshold = 0.2
y_pred_custom_threshold = (ensemble_pred >= threshold).astype(int)

# AUCスコアの計算
custom_auc = roc_auc_score(y_test, y_pred_custom_threshold)

# 結果の表示
print(f"AUC Score with custom threshold 0.5: {custom_auc:.2f}")
print("\nClassification Report with custom threshold 0.5:")
print(classification_report(y_test, y_pred_custom_threshold))

AUC Score with custom threshold 0.5: 0.72

Classification Report with custom threshold 0.5:
              precision    recall  f1-score   support

           0       0.93      0.78      0.85     10378
           1       0.35      0.67      0.46      1858

    accuracy                           0.77     12236
   macro avg       0.64      0.72      0.66     12236
weighted avg       0.84      0.77      0.79     12236



# 提出

In [257]:
y_pred_lgbm = best_lgbm.predict(test_df_encoded)
y_pred_xgb = best_xgb.predict(test_df_encoded)
y_pred_cat = best_cat.predict(test_df_encoded)

test_df_encoded.columns = test_df_encoded.columns.astype(str)
y_pred_hgb = best_hgb.predict(test_df_encoded)

ensemble_pred = (y_pred_lgbm + y_pred_xgb + y_pred_cat) / 4

threshold = 0.2
y_pred = (ensemble_pred >= threshold).astype(int)

In [258]:
# データフレームの作成
submit = pd.DataFrame(data=y_pred, index=test_df_encoded.index)
submit.index = range(40785, 40785+len(submit))
# 今日の日付の取得
today = datetime.datetime.now()
formatted_date = today.strftime("%Y%m%d")

filename = "../output/" + formatted_date + ".csv"


# csvファイルの出力
submit.to_csv(filename, index=True, header=False)